In [ ]:
print("Hello Terra!")

## Login as Pet Managed Identity

In [ ]:
!az login --identity --allow-no-subscriptions
cli_token = !az account get-access-token | jq .accessToken
token = cli_token[0].replace('"', '')

## Define some helper functions for interacting with Terra APIs
#### Note(RT): this code is illustrative; we plan to bake this functionality into client libraries

In [ ]:
import requests
DOMAIN = 'fiab-rtitle-kind-fawn.bee.envs-terra.bio'
SAM_BASE_URL = f'https://sam.{DOMAIN}'
WSM_BASE_URL = f'https://workspace.{DOMAIN}/api/workspaces/v1'
HEADERS = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

def get_sam_user():
    return requests.get(f'{SAM_BASE_URL}/register/user/v2/self/info', headers=HEADERS).json()['userEmail']

def get_workspace_sas_url():
    # Get WSM workspace id
    # Assume its the first workspace returned from WSM but we should probably have a better way.
    wsm_workspace_list = requests.get(WSM_BASE_URL, headers=HEADERS).json()
    wsm_workspace_id = wsm_workspace_list['workspaces'][0]['id']
    
    # List storage accounts in the workspace
    wsm_storage_account_list = requests.get(f'{WSM_BASE_URL}/{wsm_workspace_id}/resources', headers=HEADERS).json()
    wsm_storage_account_name = next(filter(lambda x: x['metadata']['resourceType'] == 'AZURE_STORAGE_ACCOUNT', wsm_storage_account_list['resources']))['metadata']['name']
    
    # List storage containers in the workspace
    wsm_storage_container_list = requests.get(f'{WSM_BASE_URL}/{wsm_workspace_id}/resources', headers=HEADERS).json()
    wsm_storage_container_id = next(filter(lambda x: x['metadata']['resourceType'] == 'AZURE_STORAGE_CONTAINER', wsm_storage_container_list['resources']))['metadata']['resourceId']
    
    # Get SAS URL from WSM
    wsm_sas_token_response = requests.post(f"{WSM_BASE_URL}/{wsm_workspace_id}/resources/controlled/azure/storageContainer/{wsm_storage_container_id}/getSasToken", headers=HEADERS).json()
    return wsm_sas_token_response['url']

## Print the authenticated Terra User

In [ ]:
print(get_sam_user())

## Retrieve a SAS URL for the workspace-controlled storage container

In [ ]:
sas_url = get_workspace_sas_url()
print(sas_url)

## Read/write data using SAS URL

In [ ]:
# copy this notebook file to the storage container
from urllib.parse import urlparse, urlunparse
path = urlparse(sas_url)
desired_path = path.path + '/Azure_SAS_Demo.ipynb'
blob_url = urlunparse(path._replace(path=desired_path))
! azcopy copy 'Azure_SAS_Demo.ipynb' '{blob_url}'

In [ ]:
# copy it back
import time
new_filename = 'Azure_SAS_Demo.ipynb.' + time.strftime("%Y%m%d-%H%M%S")
! azcopy copy '{blob_url}' '{new_filename}'

In [ ]:
!ls -la Azure_SAS_Demo.ipynb*